In [ ]:
#################################################################################
# Author: Paulina Arregocés Guerra
# Research groups 
# GITA (Applied Telecommunications Research Group)  - Universidad de Antioquia 
# GIDIA (Artificial Intelligence Research Group) - Universidad Nacional de Colombia
#
# Last modified on 08/06/2023 
# Desciotion: This code is used to label the dataset and to train and test the decision # tree and support vector machine models with real data. 
#################################################################################

In [ ]:
# Select your preferences and run the install command...https://pytorch.org/
#!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

# Uncomment for requirement libraries versions
#!pip install -r requirements.txt 

In [1]:
import pandas as pd
import torch
import torchvision
from sklearn.model_selection import train_test_split
import numpy as np 
import torch.optim as optim
import torch.nn as nn  # The torch.nn import gives us access to some helpful neural network things
import torch.nn.functional as F # 
import matplotlib.pyplot as plt
import ipaddress


In [3]:
#Upload Data
df_copy=pd.read_csv('.\CSV_TXT\Labeled_Data.csv')
#Divide into sets of training and testing 
df_X=df_copy[{'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol'}]
df_Y=df_copy['Label']

C:\Users\Informatica\AppData\Local\Temp\ipykernel_18340\3267101060.py:4: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_X=df_copy[{'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol'}]


In [4]:
df_X.tail()

,Dst IP,Src IP,Src Port,Dst Port,Protocol
677593,10.0.0.254,172.16.2.11,12895,53,17
677594,147.32.80.9,147.32.84.160,2180,53,17
677595,212.95.7.75,172.16.0.11,10780,53,17
677596,212.95.7.75,172.16.0.11,1138,53,17
677597,212.95.7.75,172.16.0.11,2563,53,17


In [5]:
df_Y

0         0
1         1
2         1
3         1
4         1
         ..
677593    0
677594    1
677595    0
677596    0
677597    0
Name: Label, Length: 677598, dtype: int64

In [6]:
df_Xbin=df_X.copy()
df_Ybin=df_Y

In [7]:
# We create some functions to convert real data to binary data

fix_ip=32
fix_port=16
fix_proto=8
fix_len=16
complete= 0

def ip_binary(x):
    binary_num=bin(int(ipaddress.IPv4Address(x)))
    len_binary=len(binary_num)
    if len_binary== 34:
        return binary_num[2:] 
    else:
        complete=fix_ip-(len_binary-2)
        cero='0'
        cero_complete=cero*complete 
        new_binary = binary_num.replace("0b", cero_complete)
        return new_binary

def port_binary(x):
    binary_num=bin(int(x))
    len_binary=len(binary_num)
    if len_binary== 18:
        return binary_num[2:] 
    else:
        complete=fix_port-(len_binary-2)
        cero='0'
        cero_complete=cero*complete 
        new_binary = binary_num.replace("0b", cero_complete)
        return new_binary

def proto_binary(x):
    binary_num=bin(int(x))
    len_binary=len(binary_num)
    if len_binary== 10:
        return binary_num[2:] 
    else:
        complete=fix_proto-(len_binary-2)
        cero='0'
        cero_complete=cero*complete 
        new_binary = binary_num.replace("0b", cero_complete)
        return new_binary

In [8]:
# Tranform real values to binary values

df_Xbin.loc[:,'Src IP'] = df_Xbin.loc[:,'Src IP'].apply(lambda x:ip_binary(x))
df_Xbin.loc[:,'Dst IP'] = df_Xbin.loc[:,'Dst IP'].apply(lambda x:ip_binary(x))
df_Xbin.loc[:,'Dst Port'] = df_Xbin.loc[:,'Dst Port'].apply(lambda x:port_binary(x))
df_Xbin.loc[:,'Src Port'] = df_Xbin.loc[:,'Src Port'].apply(lambda x:port_binary(x))
df_Xbin.loc[:,'Protocol'] = df_Xbin.loc[:,'Protocol'].apply(lambda x:proto_binary(x))

In [9]:
df_Xbin.tail(20)

,Dst IP,Src IP,Src Port,Dst Port,Protocol
677578,10101110100011110101001001000000,10010011001000000101010010100000,0000011000110000,0000000000011001,00000110
677579,00001010000000000000000011111110,10101100000100000000001000001011,0101011110000101,0000000000110101,00010001
677580,11010001101111110101100011111110,10010011001000000101010010110100,0000110110010001,0000000000011001,00000110
677581,11000110101001000001111000000010,11000000101010000000010101111010,0110110000110000,0000000000110101,00010001
677582,01010010011000100101011010110111,11000000101010000000001001101101,0000100100100100,0000000001010000,00000110
677583,11000111001110111001010000001010,10011110010000010110111000011000,1110011100011110,0000000001010000,00000110
677584,01001000000101000000111100111101,10010011001000000101010010110100,0000110011001000,0000000001010000,00000110
677585,10011110010000010110111000011000,01000010111110010100100100111000,1111010111100010,0000000001010000,00000110
677586,10011110010000010000110001100111,10011110010000010110111000011000,1000100110111000,0000000000110101,00010001
677587,01010011001000111111101011011101,10101100000100000000001000001100,0111011001101111,1111111101000011,00010001


In [10]:
df_XbinNN=df_Xbin.copy()
df_YbinNN=df_Ybin.copy()

In [11]:
# We create a function to obtain every single digit by feature

def Conv_bin1(string):
    vector=[]
    for i in string:
        vector.append(i)
        
    for k in range(len(vector)):
        if vector[k]=='1':vector[k]=float('1')
        elif vector[k]=='0':vector[k]=float('0')
            
    return vector

In [12]:
df_XbinNN.loc[:,'Src IP'] = df_XbinNN['Src IP'].apply(lambda x:Conv_bin1(x))
df_XbinNN.loc[:,'Dst IP'] = df_XbinNN['Dst IP'].apply(lambda x:Conv_bin1(x))
df_XbinNN.loc[:,'Dst Port'] = df_XbinNN['Dst Port'].apply(lambda x:Conv_bin1(x))
df_XbinNN.loc[:,'Src Port'] = df_XbinNN['Src Port'].apply(lambda x:Conv_bin1(x))
df_XbinNN.loc[:,'Protocol'] = df_XbinNN['Protocol'].apply(lambda x:Conv_bin1(x))

In [13]:
df_XbinNN

,Dst IP,Src IP,Src Port,Dst Port,Protocol
0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,"[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]"
2,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
3,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
4,"[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]"
...,...,...,...,...,...
677593,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
677594,"[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
677595,"[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"
677596,"[1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]"


In [14]:
# We concat every feature to create a single vector of binary data

df_XbinNN['final_vec'] = df_XbinNN['Dst Port']+df_XbinNN['Src Port']+df_XbinNN['Protocol']+df_XbinNN['Dst IP']+df_XbinNN['Src IP']

In [15]:
df_XbinNN['final_vec'].head()

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: final_vec, dtype: object

In [16]:
df_YbinNN

0         0
1         1
2         1
3         1
4         1
         ..
677593    0
677594    1
677595    0
677596    0
677597    0
Name: Label, Length: 677598, dtype: int64

In [17]:
# Uncomment to save tranform data
#df_XbinNN['final_vec'].to_csv('Datos_Trans01.csv', sep=',')
#df_YbinNN.to_csv('labels01.csv')

# Uncomment to read tranform data
#Xbin_NN=pd.read_csv('Datos_Trans01.csv')

In [18]:
# Divide train, validation and test  sets

x_trainNN, x_valNN, y_trainNN, y_valNN = train_test_split(df_XbinNN['final_vec'], df_YbinNN, train_size=0.7,random_state = 42)
x_valNN, x_testNN, y_valNN, y_testNN = train_test_split(x_valNN, y_valNN, test_size=0.5, random_state = 42)

In [19]:
x_trainNN

260249    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
563509    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
467665    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
458595    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
560268    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                                ...                        
259178    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
365838    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...
131932    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ...
671155    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
121958    [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...
Name: final_vec, Length: 474318, dtype: object

In [20]:
# Change data to torch type
target_trainNN= torch.from_numpy(np.array(y_trainNN))
target_valNN= torch.from_numpy(np.array(y_valNN))
target_testNN= torch.from_numpy(np.array(y_testNN))

features_trainNN=torch.Tensor(list(x_trainNN))
features_valNN=torch.Tensor(list(x_valNN))
features_testNN= torch.Tensor(list(x_testNN))

In [21]:
features_valNN

tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [28]:
# Save array data
'''
target_train= np.array(target_trainNN)
target_val= np.array(target_valNN)
target_test = np.array(target_testNN)
features_train = np.array(features_trainNN)
features_val = np.array(features_valNN)
features_test = np.array(features_testNN)

np.savetxt('target_train.txt', target_train, fmt='%d')
np.savetxt('target_val.txt', target_val, fmt='%d')
np.savetxt('target_test.txt', target_test, fmt='%d')
np.savetxt('features_train.txt', features_train, fmt='%d')
np.savetxt('features_val.txt', features_val, fmt='%d')
np.savetxt('features_test.txt', features_test, fmt='%d')
'''

In [29]:
# Load tensors array data
'''
a0 =torch.from_numpy(np.loadtxt('target_train.txt', dtype=int)) 
a1 =torch.from_numpy(np.loadtxt('target_val.txt', dtype=int)) 
a2 =torch.from_numpy(np.loadtxt('target_test.txt', dtype=int)) 
a3 =torch.torch.Tensor(np.loadtxt('features_train.txt', dtype=int))
a4 =torch.torch.Tensor(np.loadtxt('features_val.txt', dtype=int))
a5 =torch.torch.Tensor(np.loadtxt('features_test.txt', dtype=int)) 
'''

"a0 =torch.from_numpy(np.loadtxt('target_train.txt', dtype=int)) \na1 =torch.from_numpy(np.loadtxt('target_val.txt', dtype=int)) \na2 =torch.from_numpy(np.loadtxt('target_test.txt', dtype=int)) \na3 =torch.torch.Tensor(np.loadtxt('features_train.txt', dtype=int))\na4 =torch.torch.Tensor(np.loadtxt('features_val.txt', dtype=int))\na5 =torch.torch.Tensor(np.loadtxt('features_test.txt', dtype=int)) "

## Convert data to -1 +1 values

In [22]:
# Convertir valores
def Conv_bin(string):
    vector=[]
    for i in string:
        vector.append(i)
        
    for k in range(len(vector)):
        if vector[k]=='1':vector[k]=int('-1')
        elif vector[k]=='0':vector[k]=int('+1')
            
    return vector

def Conv_ybin(label):
    if label == 0: 
        label= +1
    elif label ==1:
        label=-1
    return label

In [23]:
df_Xneg=df_Xbin.copy()
df_Yneg=df_Ybin.copy()

In [24]:
df_Xneg.loc[:,'Src IP'] = df_Xneg['Src IP'].apply(lambda x:Conv_bin(x))
df_Xneg.loc[:,'Dst IP'] = df_Xneg['Dst IP'].apply(lambda x:Conv_bin(x))
df_Xneg.loc[:,'Dst Port'] = df_Xneg['Dst Port'].apply(lambda x:Conv_bin(x))
df_Xneg.loc[:,'Src Port'] = df_Xneg['Src Port'].apply(lambda x:Conv_bin(x))
df_Xneg.loc[:,'Protocol'] = df_Xneg['Protocol'].apply(lambda x:Conv_bin(x))

In [25]:
df_Yneg=df_Yneg.apply(lambda y:Conv_ybin(y))

In [26]:
df_Xneg.head()

,Dst IP,Src IP,Src Port,Dst Port,Protocol
0,"[1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1]"
1,"[1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1,...","[-1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1...","[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1...","[1, 1, 1, 1, 1, -1, -1, 1]"
2,"[-1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1...","[-1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1...","[1, 1, 1, -1, 1, 1, 1, -1]"
3,"[-1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1...","[-1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1...","[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, ...","[1, 1, 1, -1, 1, 1, 1, -1]"
4,"[1, -1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1...","[-1, 1, 1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1...","[1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1...","[1, 1, 1, 1, 1, -1, -1, 1]"


In [27]:
df_Xneg['final_vec'] = df_Xneg['Dst Port']+df_Xneg['Src Port']+df_Xneg['Protocol']+df_Xneg['Dst IP']+df_Xneg['Src IP']

In [28]:
df_Xneg['final_vec'].head()

0    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...
1    [1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1...
2    [1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1...
3    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, -1, ...
4    [1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1...
Name: final_vec, dtype: object

In [ ]:
# Uncomment to save data 
#df_Xneg['final_vec'].to_csv('Datos_Trans1-1.csv', sep=',')
#df_Yneg.to_csv('labels1-1.csv')

In [29]:
# Divide to train, validation and test data

x_train, x_val, y_train, y_val = train_test_split(df_Xneg['final_vec'], df_Yneg,random_state = 3, train_size=0.7)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size=0.5, random_state=42)

In [30]:
target_train= torch.from_numpy(np.array(y_train))
target_val= torch.from_numpy(np.array(y_val))
target_test= torch.from_numpy(np.array(y_test))

features_train=torch.Tensor(list(x_train))
features_val=torch.Tensor(list(x_val))
features_test= torch.Tensor(list(x_test))

In [43]:
# Uncomment to Safe array data
'''target_train_Bin= np.array(target_train)
target_val_Bin= np.array(target_val)
target_test_Bin = np.array(target_test)
features_train_Bin = np.array(features_train)
features_val_Bin = np.array(features_val)
features_test_Bin = np.array(features_test)

np.savetxt('target_train_Bin.txt', target_train_Bin, fmt='%d')
np.savetxt('target_val_Bin.txt', target_val_Bin, fmt='%d')
np.savetxt('target_test_Bin.txt', target_test_Bin, fmt='%d')
np.savetxt('features_train_Bin.txt', features_train_Bin, fmt='%d')
np.savetxt('features_val_Bin.txt', features_val_Bin, fmt='%d')
np.savetxt('features_test_Bin.txt', features_test_Bin, fmt='%d')'''


In [ ]:
# Uncomment to load tensor array data
'''
a0 =torch.from_numpy(np.loadtxt('target_train_Bin.txt', dtype=int)) 
a1 =torch.from_numpy(np.loadtxt('target_test_Bin.txt', dtype=int)) 
a2 =torch.torch.Tensor(np.loadtxt('features_train_Bin.txt', dtype=int))
a3 =torch.torch.Tensor(np.loadtxt('features_test_Bin.txt', dtype=int)) 
'''